In [1]:
from single_cell_class import *
from utils import *
from brain_seg import brain_seg_factor
from factor import thres_factor_
from brain_segment_vis import *
from swim_ephys import *
from kernel_fit import *
from scipy.stats import spearmanr
import dask.array as da
import zarr

In [2]:
df = pd.read_csv('../Processing/data_list.csv')

In [13]:
row=df.iloc[2]

In [ ]:
save_root = row['save_dir']+'/'
# check ephys data
dat_dir = row['dat_dir'].replace('/im/', '/')
dat_dir = dat_dir.replace('/im_CM0/', '/')
dat_dir = dat_dir.replace('/im_CM1/', '/')
p_dir = dat_dir + 'processed/'
ephys_dir = dat_dir + 'ephys/'
if not os.path.exists(ephys_dir):
    print('Missing directory')
    print(ephys_dir)

###################################
## Downsample sensory and motor input to frames
###################################
ephys_dat = glob(ephys_dir+'/*.10chFlt')[0]
fileContent_ = load(ephys_dat)
l_power = windowed_variance(fileContent_[0])[0]
r_power = windowed_variance(fileContent_[1])[0]
camtrig = fileContent_[2]

expt_meta = glob(dat_dir+'ephys/*end.xml')[0]
expt_paradigm = open_ephys_metadata(expt_meta)
probe_amp = (expt_paradigm.loc['LG probe']['velocity']*100).astype('int')
probe_gain = expt_paradigm.loc['LG probe']['gain']

indx = ep2frame(camtrig, thres=3.8)
frame_ = np.zeros(len(camtrig))
frame_[indx]=1
frame_ = frame_.cumsum()

slide_win = 180000
r_power_baseline = rolling_perc(r_power, window=slide_win, perc=0.1)
l_power_baseline = rolling_perc(l_power, window=slide_win, perc=0.1)

l_power_ = np.clip(l_power-l_power_baseline, 0, None)*10000
r_power_ = np.clip(r_power-r_power_baseline, 0, None)*10000

frame_len = np.min(np.unique(indx[1:]-indx[:-1]))

epoch_frame = np.median(wrap_data(fileContent_[5], indx, frame_len), axis=0).astype('int')
swim_frame = np.mean(wrap_data(l_power_, indx, frame_len), axis=0)
pulse_frame = np.rint(np.median(wrap_data(fileContent_[8], indx, frame_len), axis=0)*100).astype('int')
visu_frame = np.mean(wrap_data(fileContent_[3], indx, frame_len), axis=0)
visu_frame_ = visu_frame.copy()
visu_frame_[visu_frame_<0]=0

###################################
## Sensory cells
###################################
swim_thres = np.percentile(swim_frame, 85)
pulse_trial = []
pulse_type = []

pulse_motor_trial = []
pulse_motor_type = []

nopulse_trial = []
nopulse_type = []

pulse_amp = probe_amp #np.unique(pulse_frame)[1]
print(np.unique(pulse_frame), probe_amp)
pulse_on = np.where((pulse_frame[:-1]==0) & (pulse_frame[1:]==pulse_amp))[0]+1
num_dff = dFF.shape[-1]
pulse_on = pulse_on[pulse_on<num_dff-10]

for n, trial in enumerate(pulse_on):
    swim_ = np.clip(swim_frame[trial-2:trial+5]-swim_thres, 0, np.inf)
    if swim_.sum()==0: # remove the trial mixed pulse and motor
        pulse_trial.append(trial)
        pulse_type.append(epoch_frame[trial]//5)
    else:
        pulse_motor_trial.append(trial)
        pulse_motor_type.append(epoch_frame[trial]//5)

nopulse_on = epoch_frame%5==4
nopulse_on = np.where((~nopulse_on[:-1]) & nopulse_on[1:])[0]+1
nopulse_on = nopulse_on[nopulse_on<num_dff-10]

for n, trial in enumerate(nopulse_on):
    swim_ = np.clip(swim_frame[trial+3:trial+10]-swim_thres, 0, np.inf)
    if swim_.sum()==0:
        nopulse_trial.append(trial)
        nopulse_type.append(epoch_frame[trial]//5)
